In [1]:
# nts: activate svd 
import logging
from tqdm import tqdm
import xarray as xr
import numpy as np
import pandas as pd
import os
import sys
import contextlib
import threading
from pyproj import Proj 
from scipy.interpolate import RegularGridInterpolator
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import random 

###### settings
trackspath1='/home/sonia/mcms/tracker/1940-2010/era5/out_era5/era5/mcms_era5_1940_2010_tracks.txt'
trackspath2='/home/sonia/mcms/tracker/2010-2024/era5/out_era5/era5/FIXEDmcms_era5_2010_2024_tracks.txt'
joinyear = 2010 # overlap for the track data

use_slp = False # whether to include slp channel
use_windmag = False #include wind magnitude channel # NOTE THIS IS 500hPa
use_winduv = False # include wind u and v components channels # NOTE THIS IS 500hPa
use_temperature = False # temperature at 925hPa 
use_humidity = True # specific humidity at 500hPa
use_topo = False # include topography channel
skip_preexisting = False # skip existing datapoints (ensures they have 8 frames)
threads = 1 # >1 not implemented

# atlantic ocean is regmask['reg_name'].values[109] # so 110 in regmaskoc values
# atlantic: 110
# pacific: 111
reg_id = 110
hemi = 'n' # n or s
###### 

if reg_id == 110:
    basin = 'atlantic'
elif reg_id == 111:
    basin = 'pacific'
basin = hemi + basin

In [2]:
regmask = xr.open_dataset('/home/cyclone/regmask_0723_anl.nc')

####### make dataframe of all tracks 
tracks1 = pd.read_csv(trackspath1, sep=' ', header=None, 
        names=['year', 'month', 'day', 'hour', 'total_hrs', 'unk1', 'unk2', 'unk3', 'unk4', 'unk5', 'unk6', 
               'z1', 'z2', 'unk7', 'tid', 'sid'])
# storms that start before the join year (even if they continue into the join year):
sids1 = tracks1[(tracks1['sid']==tracks1['tid']) & (tracks1['year']<joinyear)]['sid'].unique()
tracks1 = tracks1[tracks1['sid'].isin(sids1)]

tracks2 = pd.read_csv(trackspath2, sep=' ', header=None, 
        names=['year', 'month', 'day', 'hour', 'total_hrs', 'unk1', 'unk2', 'unk3', 'unk4', 'unk5', 'unk6', 
               'z1', 'z2', 'unk7', 'tid', 'sid'])
# filter out storms that "start" at the beginning of the join year since they probably started before and are 
# included in tracks1
sids2 = tracks2[(tracks2['sid']==tracks2['tid']) & \
        ((tracks2['year']>=joinyear) | (tracks2['month']>1) | (tracks2['day']>1) | (tracks2['hour']>0))]['sid'].unique()
tracks2 = tracks2[tracks2['sid'].isin(sids2)]

tracks = pd.concat([tracks1, tracks2], ignore_index=True)
tracks = tracks.sort_values(by=['year', 'month', 'day', 'hour'])

# conversions from the MCMS lat/lon system, as described in Jimmy's email:
tracks['lat'] = 90-tracks['unk1'].values/100
tracks['lon'] = tracks['unk2'].values/100

tracks = tracks[['year', 'month', 'day', 'hour', 'tid', 'sid', 'lat', 'lon']]

In [3]:
####### variables prep
grid = 0.25
varnames = [] # list of variables that will be included in this output dataset
varlocs = {'slp': f'/mnt/data/sonia/cyclone/{grid}/slp', #'wind10m': '/home/cyclone/wind',
           'wind': f'/mnt/data/sonia/cyclone/{grid}/wind_500hpa',
           'temperature': f'/mnt/data/sonia/cyclone/{grid}/temperature',
           'humidity': f'/mnt/data/sonia/cyclone/{grid}/humidity',
           'topo': f'/mnt/data/sonia/cyclone/{grid}/slp/topo.nc'} # where the source data is stored 
varfuncs = {}
climatology = {}
if use_slp:
    varnames.append('slp')
    def f_slp(ds, lats, lons, time=None): # function to run when new SLP file is loaded
        if time is None:
            return ds.sel(lat=lats, lon=lons)['slp']
        else:
            return ds.sel(lat=lats, lon=lons, time=time)['slp']
    varfuncs['slp'] = f_slp
if use_windmag:
    varnames.append('wind')
    def f_wind(ds, lats, lons, time=None):
        if time is None:
            u = ds.sel(lat=lats, lon=lons)['u'] # for 10m: [['u10', 'v10']] 
            v = ds.sel(lat=lats, lon=lons)['v']
        else:
            u = ds.sel(lat=lats, lon=lons, time=time)['u'] # for 10m: [['u10', 'v10']] 
            v = ds.sel(lat=lats, lon=lons, time=time)['v']
        windmag = np.sqrt(u**2 + v**2)
        return windmag
    varfuncs['wind'] = f_wind
if use_winduv:
    varnames.append('wind')
    def f_winduv(ds, lats, lons, time=None):
        # print(ds.sel(lat=lats, lon=lons))
        if time is None:
            data = ds.sel(lat=lats, lon=lons)[['u', 'v']] # for 10m: [['u10', 'v10']] 
        else:
            data = ds.sel(lat=lats, lon=lons, time=time)[['u', 'v']] # for 10m: [['u10', 'v10']] 
        return data
    varfuncs['wind'] = f_winduv
if use_temperature:
    varnames.append('temperature')
    def f_temperature(ds, lats, lons, time=None):
        if time is None:
            data = ds.sel(lat=lats, lon=lons, pressure_level=925)['t']
        else:
            data = ds.sel(lat=lats, lon=lons, time=time, pressure_level=925)['t']
        return data
    varfuncs['temperature'] = f_temperature
if use_humidity:
    varnames.append('humidity')
    def f_humidity(ds, lats, lons, time=None):
        if time is None:
            data = ds.sel(lat=lats, lon=lons, pressure_level=500)['q']
        else:
            data = ds.sel(lat=lats, lon=lons, time=time, pressure_level=500)['q']
        return data 
    varfuncs['humidity'] = f_humidity
topo = None
if use_topo: 
    varnames.append('topo')
    topo = xr.open_dataset(varlocs['topo'], engine='netcdf4')
    def f_topo(ds, lats, lons, time=None):
        return ds.sel(lat=lats, lon=lons)['lsm']
varnames, varfuncs

resolution = grid # resolution of data in degs (may later get redefined by climax checkpoint reso)
l = 800 # (half length: l/2 km from center in each direction)
s = 32 # box will be dimensions s by s (eg 32x32)
x_lin = np.linspace(-l, l, s)
y_lin = np.linspace(-l, l, s)
x_grid, y_grid = np.meshgrid(x_lin, y_lin) # equal-spaced points from -l to l in both x and y dimensions

In [4]:
file_year = 1940
end_year = 2024
cur_datas = {}
for var in varnames:
    cur_data = xr.open_dataset(f'{varlocs[var]}/{var}.{file_year}.nc', engine='netcdf4')
    correct_time = cur_data['time'].values[0] + pd.to_timedelta(np.arange(cur_data.dims['time']) * 6, unit='h')
    cur_data = cur_data.assign_coords(time=correct_time) # incase it wasn't read in as 6hrly
    cur_datas[var] = cur_data

/tmp/ipykernel_3519284/1857287072.py:6: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  correct_time = cur_data['time'].values[0] + pd.to_timedelta(np.arange(cur_data.dims['time']) * 6, unit='h')


In [5]:
truetrain = set(os.listdir(f'/home/cyclone/train/windmag/500hpa/0.25/date/{basin}/train'))
trueval = set(os.listdir(f'/home/cyclone/train/windmag/500hpa/0.25/date/{basin}/val'))
truetest = set(os.listdir(f'/home/cyclone/train/windmag/500hpa/0.25/date/{basin}/test'))

tracks['split'] = 0
tracks.loc[tracks['sid'].isin(trueval), 'split'] = 1
tracks.loc[tracks['sid'].isin(truetest), 'split'] = 2

# The inverse
From global climaX output to series of 32x32 matrices

In [6]:
from pyproj import Proj 
from scipy.interpolate import RegularGridInterpolator
predpath = '/home/sonia/climaxinf/out/date/natlantic-multivar-fullcontext/test'
outpath = '/mnt/data/sonia/climax-data/date/extracted-natlantic-multivar-fullcontext/test'
os.makedirs(outpath, exist_ok=True)

resolution = 1.40625
l = 800 # (half length: l/2 km from center in each direction)
s = 32 # box will be dimensions s by s (eg 32x32)
### HERE WE REINTRODUE THE N/S FLIP: ###
x_lin = np.linspace(-l, l, s)
y_lin = np.linspace(-l, l, s)
x_grid, y_grid = np.meshgrid(x_lin, y_lin) # equal-spaced points from -l to l in both x and y dimensions

In [8]:
for fname in tqdm(os.listdir(predpath)):
    worlds = np.load(os.path.join(predpath, fname))
    if np.isnan(worlds).sum() > 0:
        print('NANS', np.isnan(worlds).sum())
    sid = fname[:-4]
    records = tracks[tracks['sid']==sid].to_dict('records')
    boxes = []
        
    for record, world in zip(records, worlds): # iterates over time
        lats = np.linspace(-90, 90, 128)
        lons = np.linspace(0, 360, 256, endpoint=False)
        ds = xr.Dataset(
            data_vars={'t2m': (('lat', 'lon'), world[0]), # EDIT THESE!
                       'z': (('lat', 'lon'), world[1]),
                       'u': (('lat', 'lon'), world[2]),
                       'v': (('lat', 'lon'), world[3]),
                       't': (('lat', 'lon'), world[4]),
                       'q': (('lat', 'lon'), world[5])}, 
            coords={"lat": lats, "lon": lons}, # Attach the coordinates
        )

        lat_center, lon_center = record['lat'], record['lon']
        proj_km = Proj(proj='aeqd', lat_0=lat_center, lon_0=lon_center, units='km')
        lon_grid, lat_grid = proj_km(x_grid, y_grid, inverse=True) #translate km to deg
        lon_grid=(lon_grid+360)%360 # because these datasets have lon as 0 to 360 (lat is still -90 to 90)
        lon_min = lon_grid.min() - resolution # +- reso because otherwise xarray will not include the edge points
        lon_max = lon_grid.max() + resolution
        lat_min = lat_grid.min() - resolution
        lat_max = lat_grid.max() + resolution

        selection = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
        lats = selection.lat.values 
        lons = selection.lon.values
        data = selection.to_array().values

        slices = []
        if data.shape[0] > 1: # for instance, wind u and v components
            for i in range(data.shape[0]):
                # Build interpolator
                interp = RegularGridInterpolator(
                    (lats, lons),
                    data[i],
                    bounds_error=False,
                    fill_value=None
                )

                # Interpolate at new (lat, lon) pairs
                interp_points = np.stack([lat_grid.ravel(), lon_grid.ravel()], axis=-1)
                interp_values = interp(interp_points).reshape(s, s)
                slices.append(interp_values)
        else:
            # Build interpolator
            interp = RegularGridInterpolator(
                (lats, lons),
                data.squeeze(),
                bounds_error=False,
                fill_value=np.nan
            )

            # Interpolate at new (lat, lon) pairs
            interp_points = np.stack([lat_grid.ravel(), lon_grid.ravel()], axis=-1)
            interp_values = interp(interp_points).reshape(s, s)
            slices.append(interp_values)
            
        # windmag = np.sqrt(slices[0]**2 + slices[1]**2)
        # boxes.append(windmag)
        slp = 925 * np.exp(slices[1] / (287.05 * slices[0])) # Formula: P_slp = P_level * exp(Phi / (Rd * T))
        frame = np.stack([slp]+slices[2:], axis=-1) # We want H x W x V
        boxes.append(frame)

    # result = np.stack(boxes, axis=0)
    # np.save(os.path.join(outpath, f'{sid}.npy'), result)
    os.makedirs(os.path.join(outpath, sid), exist_ok=True)
    for i in range(len(boxes)):
        np.save(os.path.join(outpath, sid, f'{i}.npy'), boxes[i])

100%|██████████| 899/899 [01:47<00:00,  8.35it/s]


In [15]:
boxes[0].shape

(32, 32, 5)

In [16]:
worlds.shape

(8, 6, 128, 256)

# Climatology

In [22]:
out_path = f'/mnt/data/sonia/clima_patches/date/{basin}-humidity-0.25'
climatology_path = '/mnt/data/sonia/cyclone/0.25/humidity/monthly_climatology.nc'
clima = xr.open_dataset(climatology_path)

os.makedirs(out_path, exist_ok=True)
readme = f'based on climatology at {climatology_path}'
with open(os.path.join(out_path, 'readme.txt'), 'w') as f:
    f.write(readme)

testvalsids = tracks[tracks['split']>0]
testvalframes = [group.to_dict('records') for _, group in testvalsids.groupby('sid')]
out_path

'/mnt/data/sonia/clima_patches/date/natlantic-humidity-0.25'

In [23]:
def prep_point_fulldata(frames): # provide actual climate data, not patch plus climatology
    """make one training datapoint. df contains year/../hr, lat, lon of center"""
    # if skip_preexisting and frame['sid'] in exists:
    #     return
    boxes = []
            
    for frame in frames:
        year, month, day, hour = frame['year'], frame['month'], frame['day'], frame['hour']
        time = f'{year}-{month:02d}-{day:02d}T{hour:02d}:00:00'
        cur_datas = clima.sel(month=month)
        
        lat_center, lon_center = frame['lat'], frame['lon']
        # 'aeqd': https://proj.org/en/stable/operations/projections/aeqd.html
        proj_km = Proj(proj='aeqd', lat_0=lat_center, lon_0=lon_center, units='km')
        # Project to find lat/lon corners of the equal-area box
        lon_grid, lat_grid = proj_km(x_grid, y_grid, inverse=True) #translate km to deg
        lon_grid=(lon_grid+360)%360 # because these datasets have lon as 0 to 360 (lat is still -90 to 90)
        lon_min = lon_grid.min() - resolution # +- reso because otherwise xarray will not include the edge points
        lon_max = lon_grid.max() + resolution
        lat_min = lat_grid.min() - resolution
        lat_max = lat_grid.max() + resolution
        
        # for var in varnames:
        data = varfuncs[var](cur_datas, slice(lat_max, lat_min), slice(lon_min, lon_max), time=None)
        
        lats = data.lat.values 
        lons = data.lon.values
        
        slices=[]
        if type(data) == xr.Dataset: # for instance, wind u and v components (data.shape[0] or data.to_array().shape[0] ??)
            data = data.to_array().values.squeeze()
            # print(data.shape)
            for i in range(data.shape[0]):
                sel = data[i]
                # Build interpolator
                interp = RegularGridInterpolator(
                    (lats, lons),
                    sel,
                    bounds_error=False,
                    fill_value=np.nan
                )
                
                # Interpolate at new (lat, lon) pairs
                interp_points = np.stack([lat_grid.ravel(), lon_grid.ravel()], axis=-1)
                interp_values = interp(interp_points).reshape(s, s)
                slices.append(interp_values)
        else: # just one channel (eg slp)
            data = np.asarray(data).squeeze()
            # Build interpolator
            interp = RegularGridInterpolator(
                (lats, lons),
                data,
                bounds_error=False,
                fill_value=np.nan
            )
            
            # Interpolate at new (lat, lon) pairs
            interp_points = np.stack([lat_grid.ravel(), lon_grid.ravel()], axis=-1)
            interp_values = interp(interp_points).reshape(s, s)
            slices.append(interp_values)
            
        boxes.append(np.stack(slices, axis=-1).squeeze())
        
    # split = '' 
    if frame['split'] == 0:
        split = 'train'
    elif frame['split'] == 1:
        split = 'val'
    elif frame['split'] == 2:
        split = 'test'
    else:
        raise ValueError(f"Unexpected split value {frame['split']}")
    
    os.makedirs(os.path.join(out_path, split, frame['sid']), exist_ok=True)
    for i in range(len(boxes)):
        np.save(os.path.join(out_path, split, f"{frame['sid']}/{i}"), boxes[i])
        
    return boxes
    
# prompt = prep_point(tracks.iloc[0])

In [24]:
os.makedirs(os.path.join(out_path, 'val'), exist_ok=True)
os.makedirs(os.path.join(out_path, 'test'), exist_ok=True)

for storm in tqdm(testvalframes):
    boxes =prep_point_fulldata(storm[:8])

100%|██████████| 899/899 [00:08<00:00, 109.94it/s]


In [44]:
boxes[0].shape

(32, 32)